In [45]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import re
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

In [10]:
df = pd.read_csv('F:\Fifth Year\\First Term\\NLP\\dataset\\1\\Laptop_Train_v2.csv')
df

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12
...,...,...,...,...,...,...
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170
2355,848,"How Toshiba handles the repair seems to vary, ...",repair,conflict,24,30
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136


In [11]:
aspect_term = df["Aspect Term"].tolist()
aspect_term_word = word_tokenize(" ".join(aspect_term))


def reomve_unnecessary_word(List):
    unnecessary_word = {}

    for word, freq in List:
        if word not in aspect_term_word:
            unnecessary_word[word] = freq
    
    return unnecessary_word

In [12]:
df["polarity"] = df["polarity"].replace({"neutral": 0, "negative": -1, "positive": 1})
df = df[(df["polarity"] == 1) | (df["polarity"] == 0) | (df["polarity"] == -1)]

In [13]:
# Group by "id" and "Sentence" and sum the polarity values
df_grouped = df.groupby(["id", "Sentence"]).agg({"polarity": "sum"}).reset_index()

# Rename the "polarity" column to "sentiment"
df_grouped = df_grouped.rename(columns={"polarity": "sentiment"})

# Drop duplicate rows based on "id" and keep the first occurrence
df_grouped = df_grouped.drop_duplicates(subset=["id"])

# Reset the index of the DataFrame
df_grouped = df_grouped.reset_index(drop=True)

df = df_grouped
df

,id,Sentence,sentiment
0,3,It's so nice to look at and the keys are easy ...,2
1,4,I bought a protector for my key pad and it wor...,1
2,7,AND the best part is that it even comes with a...,1
3,8,"Overall, this laptop is definitely a keeper wi...",2
4,9,The Apple team also assists you very nicely wh...,1
...,...,...,...
1457,3080,I wish Toshiba would allow their customers to ...,0
1458,3081,Once I removed all the software the laptop loa...,-1
1459,3082,This is not a serious gaming laptop or a serio...,-1
1460,3083,my niece and nephew have played a few web game...,0


In [14]:
dist = FreqDist(sum(df['Sentence'].map(word_tokenize), []))
dist

FreqDist({'.': 1335, 'the': 1268, ',': 1008, 'and': 810, 'to': 727, 'I': 696, 'a': 578, 'it': 567, 'is': 531, 'of': 363, ...})

In [15]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [16]:
# # Filter the dataset based on the polarity column
# positive_sentences = df[df["polarity"] == "positive"]["Sentence"]
# negative_sentences = df[df["polarity"] == "negative"]["Sentence"]
# neutral_sentences = df[df["polarity"] == "neutral"]["Sentence"]

# Filter the dataset based on the polarity column
positive_sentences = df[df["sentiment"] >= 1]["Sentence"]
negative_sentences = df[df["sentiment"] <= -1]["Sentence"]
neutral_sentences = df[df["sentiment"] == 0]["Sentence"]

# Tokenize and calculate the frequency distribution for positive sentences
positive_words = word_tokenize(" ".join(positive_sentences))
positive_dist = FreqDist(positive_words)

# Tokenize and calculate the frequency distribution for negative sentences
negative_words = word_tokenize(" ".join(negative_sentences))
negative_dist = FreqDist(negative_words)

# Tokenize and calculate the frequency distribution for neutral sentences
neutral_words = word_tokenize(" ".join(neutral_sentences))
neutral_dist = FreqDist(neutral_words)

# Print the frequency distributions
print("Positive Sentences Frequency Distribution:")
print(positive_dist)

print("\nNegative Sentences Frequency Distribution:")
print(negative_dist)

print("\nNeutral Sentences Frequency Distribution:")
print(neutral_dist)

Positive Sentences Frequency Distribution:
<FreqDist with 1987 samples and 11337 outcomes>

Negative Sentences Frequency Distribution:
<FreqDist with 2339 samples and 12754 outcomes>

Neutral Sentences Frequency Distribution:
<FreqDist with 1257 samples and 4528 outcomes>


In [17]:
remove_stopWord_positive_words = [(word, freq) for word, freq in positive_dist.items() if word.lower() not in stop_words]
remove_stopWord_positive_words

[("'s", 42),
 ('nice', 16),
 ('look', 11),
 ('keys', 5),
 ('easy', 70),
 ('type', 5),
 ('.', 548),
 ('bought', 11),
 ('protector', 1),
 ('key', 5),
 ('pad', 9),
 ('works', 27),
 ('great', 88),
 (':', 13),
 (')', 46),
 ('best', 16),
 ('part', 2),
 ('even', 21),
 ('comes', 13),
 ('free', 6),
 ('printer', 3),
 ('(', 46),
 ('certain', 2),
 ('promotion/offer', 1),
 ('going', 2),
 (',', 446),
 ('course', 2),
 ('!', 49),
 ('Overall', 3),
 ('laptop', 64),
 ('definitely', 6),
 ('keeper', 1),
 ('simple', 7),
 ('yet', 5),
 ('stylish', 3),
 ('design', 14),
 ('array', 1),
 ('fantastic', 4),
 ('colors', 3),
 ('choose', 2),
 ('Apple', 18),
 ('team', 1),
 ('also', 34),
 ('assists', 1),
 ('nicely', 2),
 ('choosing', 1),
 ('computer', 63),
 ('right', 7),
 ('[', 1),
 ('$', 13),
 (']', 1),
 ('price', 43),
 ('special', 2),
 ('offer', 2),
 ('much', 24),
 ('``', 6),
 ('junk', 1),
 ("''", 13),
 ('software', 23),
 ('remove', 1),
 ('Runs', 2),
 ('fast', 29),
 ('regular', 3),
 ('layout', 1),
 ('keyboard', 28),
 

In [18]:
remove_stopWord_negative_words = [(word, freq) for word, freq in negative_dist.items() if word.lower() not in stop_words]
remove_stopWord_negative_words

[('adjust', 1),
 ('mousepad', 3),
 ('sensitivity', 2),
 (',', 386),
 ('sensitive', 2),
 ('.', 572),
 ('First', 5),
 ('push', 2),
 ('button', 6),
 ('open', 5),
 ('lid', 2),
 ('leading', 1),
 ('edge', 2),
 ('razor', 1),
 ('sharp', 3),
 ('complete', 2),
 ('opposite', 1),
 ('ergonomic', 1),
 ('design', 4),
 ('much', 10),
 ('lauded', 1),
 ('combined', 1),
 ('touch', 3),
 ('pad', 10),
 ('clicker', 1),
 ('nightmare', 3),
 ('Also', 12),
 ('space', 3),
 ('bar', 2),
 ('makes', 6),
 ('noisy', 1),
 ('click', 3),
 ('every', 10),
 ('time', 29),
 ('use', 29),
 ('absolutely', 1),
 ('horrible', 4),
 ('despite', 1),
 ('called', 9),
 ('advanced', 1),
 ('features', 4),
 ('regret', 1),
 ('buying', 2),
 ('understanding', 1),
 ('awful', 5),
 ('due', 12),
 ('course', 7),
 ("'ll", 4),
 ('remove', 2),
 ('hard', 31),
 ('disc', 4),
 ('new', 20),
 ('MacBook', 5),
 ('Pro', 4),
 ('dump', 1),
 ('belongs', 1),
 ('-', 8),
 ('trash', 1),
 ('computer', 72),
 ('runs', 1),
 ('extremely', 3),
 ('slowly', 3),
 ('whether', 1)

In [19]:
remove_stopWord_neutral_words = [(word, freq) for word, freq in neutral_dist.items() if word.lower() not in stop_words]
remove_stopWord_neutral_words

[('AMD', 2),
 ('Turin', 1),
 ('processor', 2),
 ('seems', 4),
 ('always', 4),
 ('perform', 1),
 ('much', 6),
 ('better', 7),
 ('Intel', 3),
 ('.', 209),
 ('shop', 2),
 (',', 176),
 ('MacBooks', 1),
 ('encased', 1),
 ('soft', 1),
 ('rubber', 1),
 ('enclosure', 1),
 ('-', 3),
 ('never', 2),
 ('know', 4),
 ('razor', 1),
 ('edge', 1),
 ('buy', 8),
 ('get', 9),
 ('home', 1),
 ('break', 1),
 ('seal', 1),
 ('use', 14),
 ('(', 21),
 ('clever', 1),
 ('con', 1),
 (')', 22),
 ('Whenever', 1),
 ('call', 5),
 ('Dell', 2),
 ('unrelated', 1),
 ('problem', 9),
 ('ask', 2),
 ('laptop', 21),
 ('running', 3),
 ('slowly', 1),
 ("'d", 4),
 ('like', 17),
 ('purchase', 7),
 ('memory', 4),
 ('$', 9),
 ('75', 1),
 ('OS', 5),
 ('takes', 4),
 ('getting', 6),
 ('used', 10),
 ('especially', 2),
 ('Windows', 20),
 ('user', 2),
 ('long', 2),
 ('learning', 2),
 ('curve', 1),
 ('worth', 4),
 ('!', 8),
 ('course', 4),
 ('bought', 6),
 ('3', 9),
 ('year', 7),
 ('warranty', 23),
 ('sent', 6),
 ('replaced', 7),
 ('almost'

In [20]:
reg_positive_words = [(word, freq) for word, freq in remove_stopWord_positive_words if re.match('^[a-z_]+$',word)]
reg_positive_words

[('nice', 16),
 ('look', 11),
 ('keys', 5),
 ('easy', 70),
 ('type', 5),
 ('bought', 11),
 ('protector', 1),
 ('key', 5),
 ('pad', 9),
 ('works', 27),
 ('great', 88),
 ('best', 16),
 ('part', 2),
 ('even', 21),
 ('comes', 13),
 ('free', 6),
 ('printer', 3),
 ('certain', 2),
 ('going', 2),
 ('course', 2),
 ('laptop', 64),
 ('definitely', 6),
 ('keeper', 1),
 ('simple', 7),
 ('yet', 5),
 ('stylish', 3),
 ('design', 14),
 ('array', 1),
 ('fantastic', 4),
 ('colors', 3),
 ('choose', 2),
 ('team', 1),
 ('also', 34),
 ('assists', 1),
 ('nicely', 2),
 ('choosing', 1),
 ('computer', 63),
 ('right', 7),
 ('price', 43),
 ('special', 2),
 ('offer', 2),
 ('much', 24),
 ('junk', 1),
 ('software', 23),
 ('remove', 1),
 ('fast', 29),
 ('regular', 3),
 ('layout', 1),
 ('keyboard', 28),
 ('better', 16),
 ('longer', 3),
 ('battery', 43),
 ('life', 35),
 ('would', 19),
 ('meets', 3),
 ('spec', 2),
 ('quite', 3),
 ('easily', 12),
 ('people', 4),
 ('say', 4),
 ('hold', 2),
 ('functionality', 1),
 ('value',

In [21]:
reg_negative_words = [(word, freq) for word, freq in remove_stopWord_negative_words if re.match('^[a-z_]+$',word)]
reg_negative_words

[('adjust', 1),
 ('mousepad', 3),
 ('sensitivity', 2),
 ('sensitive', 2),
 ('push', 2),
 ('button', 6),
 ('open', 5),
 ('lid', 2),
 ('leading', 1),
 ('edge', 2),
 ('razor', 1),
 ('sharp', 3),
 ('complete', 2),
 ('opposite', 1),
 ('ergonomic', 1),
 ('design', 4),
 ('much', 10),
 ('lauded', 1),
 ('combined', 1),
 ('touch', 3),
 ('pad', 10),
 ('clicker', 1),
 ('nightmare', 3),
 ('space', 3),
 ('bar', 2),
 ('makes', 6),
 ('noisy', 1),
 ('click', 3),
 ('every', 10),
 ('time', 29),
 ('use', 29),
 ('absolutely', 1),
 ('horrible', 4),
 ('despite', 1),
 ('called', 9),
 ('advanced', 1),
 ('features', 4),
 ('regret', 1),
 ('buying', 2),
 ('understanding', 1),
 ('awful', 5),
 ('due', 12),
 ('course', 7),
 ('remove', 2),
 ('hard', 31),
 ('disc', 4),
 ('new', 20),
 ('dump', 1),
 ('belongs', 1),
 ('trash', 1),
 ('computer', 72),
 ('runs', 1),
 ('extremely', 3),
 ('slowly', 3),
 ('whether', 1),
 ('opening', 2),
 ('like', 22),
 ('laptop', 46),
 ('run', 7),
 ('well', 12),
 ('without', 7),
 ('purchase', 

In [22]:
reg_neutral_words = [(word, freq) for word, freq in remove_stopWord_neutral_words if re.match('^[a-z_]+$',word)]
reg_neutral_words

[('processor', 2),
 ('seems', 4),
 ('always', 4),
 ('perform', 1),
 ('much', 6),
 ('better', 7),
 ('shop', 2),
 ('encased', 1),
 ('soft', 1),
 ('rubber', 1),
 ('enclosure', 1),
 ('never', 2),
 ('know', 4),
 ('razor', 1),
 ('edge', 1),
 ('buy', 8),
 ('get', 9),
 ('home', 1),
 ('break', 1),
 ('seal', 1),
 ('use', 14),
 ('clever', 1),
 ('con', 1),
 ('call', 5),
 ('unrelated', 1),
 ('problem', 9),
 ('ask', 2),
 ('laptop', 21),
 ('running', 3),
 ('slowly', 1),
 ('like', 17),
 ('purchase', 7),
 ('memory', 4),
 ('takes', 4),
 ('getting', 6),
 ('used', 10),
 ('especially', 2),
 ('user', 2),
 ('long', 2),
 ('learning', 2),
 ('curve', 1),
 ('worth', 4),
 ('course', 4),
 ('bought', 6),
 ('year', 7),
 ('warranty', 23),
 ('sent', 6),
 ('replaced', 7),
 ('almost', 1),
 ('months', 4),
 ('later', 2),
 ('replacement', 3),
 ('partially', 1),
 ('due', 1),
 ('fact', 2),
 ('keyboard', 10),
 ('generally', 1),
 ('felt', 1),
 ('okay', 1),
 ('even', 6),
 ('someone', 2),
 ('desktop', 6),
 ('looks', 1),
 ('terri

In [23]:
cleaned_positive_words = reomve_unnecessary_word(reg_positive_words)
cleaned_negative_words = reomve_unnecessary_word(reg_negative_words)
cleaned_neutral_words = reomve_unnecessary_word(reg_neutral_words)

In [24]:
# Init the stemmer
stemmer = SnowballStemmer('english')

stemmed_positive_dict = {}

for word, freq in cleaned_positive_words.items():
    stemmed_word = stemmer.stem(word)  # Apply stemming to the word
    if stemmed_word in stemmed_positive_dict:
        stemmed_positive_dict[stemmed_word] += freq  # Increment the frequency if the stemmed word already exists
    else:
        stemmed_positive_dict[stemmed_word] = freq  # Add the stemmed word with its frequency to the dictionary
        
        
print("Befor Stemming: ", len(reg_positive_words))
print("After Stemming: ", len(stemmed_positive_dict))

Befor Stemming:  1450
After Stemming:  887


In [25]:
stemmed_negative_dict = {}

for word, freq in cleaned_negative_words.items():
    stemmed_word = stemmer.stem(word)  # Apply stemming to the word
    if stemmed_word in stemmed_negative_dict:
        stemmed_negative_dict[stemmed_word] += freq  # Increment the frequency if the stemmed word already exists
    else:
        stemmed_negative_dict[stemmed_word] = freq  # Add the stemmed word with its frequency to the dictionary
        
        
print("Befor Stemming: ", len(reg_negative_words))  
print("After Stemming: ", len(stemmed_negative_dict))

Befor Stemming:  1658
After Stemming:  1016


In [26]:
stemmed_neutral_dict = {}

for word, freq in cleaned_neutral_words.items():
    stemmed_word = stemmer.stem(word)  # Apply stemming to the word
    if stemmed_word in stemmed_neutral_dict:
        stemmed_neutral_dict[stemmed_word] += freq  # Increment the frequency if the stemmed word already exists
    else:
        stemmed_neutral_dict[stemmed_word] = freq  # Add the stemmed word with its frequency to the dictionary
       
    
print("Befor Stemming: ", len(reg_neutral_words))
print("After Stemming: ", len(stemmed_neutral_dict))

Befor Stemming:  887
After Stemming:  540


In [27]:
stemmed_positive = [(word, freq) for word, freq in stemmed_positive_dict.items() if freq > 10]
stemmed_positive

[('nice', 18),
 ('easi', 70),
 ('bought', 11),
 ('great', 88),
 ('best', 16),
 ('even', 21),
 ('come', 19),
 ('laptop', 76),
 ('also', 34),
 ('much', 24),
 ('fast', 29),
 ('better', 16),
 ('would', 19),
 ('easili', 12),
 ('love', 34),
 ('need', 28),
 ('amaz', 15),
 ('like', 37),
 ('mani', 12),
 ('new', 20),
 ('good', 42),
 ('thing', 24),
 ('well', 33),
 ('awesom', 13),
 ('get', 19),
 ('use', 16),
 ('machin', 14),
 ('enough', 17),
 ('perfect', 14),
 ('hour', 11),
 ('long', 14),
 ('problem', 13),
 ('want', 13),
 ('make', 20),
 ('realli', 18),
 ('quick', 11),
 ('excel', 12),
 ('take', 21),
 ('lot', 16),
 ('abl', 12),
 ('purchas', 11),
 ('alway', 12),
 ('open', 11)]

In [28]:
stemmed_negative = [(word, freq) for word, freq in stemmed_negative_dict.items() if freq > 10]
stemmed_negative

[('complet', 14),
 ('make', 24),
 ('call', 13),
 ('buy', 21),
 ('due', 12),
 ('new', 20),
 ('like', 24),
 ('laptop', 50),
 ('well', 12),
 ('purchas', 17),
 ('would', 59),
 ('also', 16),
 ('replac', 26),
 ('littl', 14),
 ('still', 18),
 ('never', 11),
 ('hour', 20),
 ('month', 35),
 ('stop', 13),
 ('less', 12),
 ('machin', 14),
 ('slow', 13),
 ('crash', 13),
 ('week', 16),
 ('get', 48),
 ('way', 18),
 ('time', 12),
 ('send', 16),
 ('thing', 28),
 ('see', 12),
 ('sinc', 16),
 ('back', 34),
 ('got', 24),
 ('problem', 49),
 ('bad', 18),
 ('bought', 13),
 ('want', 11),
 ('pay', 13),
 ('issu', 13),
 ('say', 11),
 ('could', 22),
 ('chang', 12),
 ('come', 17),
 ('tri', 14),
 ('fix', 22),
 ('turn', 11),
 ('use', 11),
 ('think', 11),
 ('good', 12),
 ('go', 15),
 ('netbook', 17),
 ('differ', 11),
 ('even', 28),
 ('sent', 11),
 ('first', 13),
 ('keep', 13),
 ('went', 17),
 ('ca', 12)]

In [29]:
stemmed_neutral = [(word, freq) for word, freq in stemmed_neutral_dict.items() if freq > 10]
stemmed_neutral

[('get', 17),
 ('problem', 13),
 ('laptop', 27),
 ('like', 17),
 ('purchas', 12),
 ('use', 15),
 ('want', 13),
 ('need', 15),
 ('would', 12)]

In [42]:
def count_pos(text):
    pos = 0
    for w in word_tokenize(text):
        if any(stemmer.stem(w) == pair[0] for pair in stemmed_positive):
            pos +=1
            
    return pos
        
def count_neg(text):
    neg = 0
    for w in word_tokenize(text):
        if any(stemmer.stem(w) == pair[0] for pair in stemmed_negative):
            neg += 1

    return neg

def count_neu(text):
    neu = 0
    for w in word_tokenize(text):
        if any(stemmer.stem(w) == pair[0] for pair in stemmed_neutral):
            neu +=1

    return neu

def pre_process(df):
    df['positive'] = df['Sentence'].apply(count_pos)
    df['negative'] = df['Sentence'].apply(count_neg)
    df['neutral'] = df['Sentence'].apply(count_neu)
    return df
pre_process(df)

,id,Sentence,sentiment,positive,negative,neutral
0,3,It's so nice to look at and the keys are easy ...,2,2,0,0
1,4,I bought a protector for my key pad and it wor...,1,2,1,0
2,7,AND the best part is that it even comes with a...,1,3,3,0
3,8,"Overall, this laptop is definitely a keeper wi...",2,1,1,1
4,9,The Apple team also assists you very nicely wh...,1,2,1,0
...,...,...,...,...,...,...
1457,3080,I wish Toshiba would allow their customers to ...,0,1,1,1
1458,3081,Once I removed all the software the laptop loa...,-1,1,1,1
1459,3082,This is not a serious gaming laptop or a serio...,-1,2,2,1
1460,3083,my niece and nephew have played a few web game...,0,0,0,0


In [43]:
X = df[['positive', 'negative','neutral']]

In [46]:
df['sentiment'] = np.where(df['sentiment'] >= 1, 1, np.where(df['sentiment'] <= -1, -1, 0))

In [47]:
df

,id,Sentence,sentiment,positive,negative,neutral
0,3,It's so nice to look at and the keys are easy ...,1,2,0,0
1,4,I bought a protector for my key pad and it wor...,1,2,1,0
2,7,AND the best part is that it even comes with a...,1,3,3,0
3,8,"Overall, this laptop is definitely a keeper wi...",1,1,1,1
4,9,The Apple team also assists you very nicely wh...,1,2,1,0
...,...,...,...,...,...,...
1457,3080,I wish Toshiba would allow their customers to ...,0,1,1,1
1458,3081,Once I removed all the software the laptop loa...,-1,1,1,1
1459,3082,This is not a serious gaming laptop or a serio...,-1,2,2,1
1460,3083,my niece and nephew have played a few web game...,0,0,0,0


In [49]:
from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
y = l.fit_transform(df[['sentiment']])
y

C:\Users\dell\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([2, 2, 2, ..., 0, 1, 1], dtype=int64)

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
X_train , X_test, y_train , y_test = train_test_split(X,y, test_size=0.2)

In [52]:
X_train

,positive,negative,neutral
1024,0,0,0
167,1,1,0
83,3,0,0
301,1,1,1
547,6,6,3
...,...,...,...
417,0,0,0
1115,2,1,0
1223,1,1,0
1118,1,0,0


In [53]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

In [54]:
clf.fit(X_train , y_train)

DecisionTreeClassifier()

In [55]:
y_predict = clf.predict(X_test)
y_predict

array([2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 1, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2,
       2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0,
       0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2,
       2, 0, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 1, 1, 2, 2, 0, 0, 2, 2,
       2, 1, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2,
       0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 1, 2, 0, 0, 2, 0,
       0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2,
       2, 0, 0, 2, 2, 1, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2,
       1, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0,
       2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0,
       0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2,
       0, 2, 2, 2, 0, 0, 0], dtype=int64)

In [56]:
clf

DecisionTreeClassifier()

In [60]:
from sklearn.metrics import accuracy_score, f1_score, precision_score , recall_score

print('accuracy_score', accuracy_score(y_test, y_predict))
print('f1_score' , f1_score(y_test, y_predict, average='weighted'))
print('precision_score', precision_score(y_test, y_predict, average='weighted'))
print('recall_score' , recall_score(y_test, y_predict, average='weighted'))

accuracy_score 0.6040955631399317
f1_score 0.5627482286231357
precision_score 0.5881068630133244
recall_score 0.6040955631399317
